# Plan your trip with Kayak 

## Company's description 📇

<a href="https://www.kayak.com" target="_blank">Kayak</a> is a travel search engine that helps user plan their next trip at the best price.

The company was founded in 2004 by Steve Hafner & Paul M. English. After a few rounds of fundraising, Kayak was acquired by <a href="https://www.bookingholdings.com/" target="_blank">Booking Holdings</a> which now holds: 

* <a href="https://booking.com/" target="_blank">Booking.com</a>
* <a href="https://kayak.com/" target="_blank">Kayak</a>
* <a href="https://www.priceline.com/" target="_blank">Priceline</a>
* <a href="https://www.agoda.com/" target="_blank">Agoda</a>
* <a href="https://Rentalcars.com/" target="_blank">RentalCars</a>
* <a href="https://www.opentable.com/" target="_blank">OpenTable</a>

With over \$300 million revenue a year, Kayak operates in almost all countries and all languages to help their users book travels accros the globe. 

## Project 🚧

The marketing team needs help on a new project. After doing some user research, the team discovered that **70% of their users who are planning a trip would like to have more information about the destination they are going to**. 

In addition, user research shows that **people tend to be defiant about the information they are reading if they don't know the brand** which produced the content. 

Therefore, Kayak Marketing Team would like to create an application that will recommend where people should plan their next holidays. The application should be based on real data about:

* Weather 
* Hotels in the area 

The application should then be able to recommend the best destinations and hotels based on the above variables at any given time. 

## Goals 🎯

As the project has just started, your team doesn't have any data that can be used to create this application. Therefore, your job will be to: 

* Scrape data from destinations 
* Get weather data from each destination 
* Get hotels' info about each destination
* Store all the information above in a data lake
* Extract, transform and load cleaned data from your datalake to a data warehouse

## Scope of this project 🖼️

Marketing team wants to focus first on the best cities to travel to in France. According <a href="https://one-week-in.com/35-cities-to-visit-in-france/" target="_blank">One Week In.com</a> here are the top-35 cities to visit in France: 

### Get weather data with an API 

*   Use https://nominatim.org/ to get the gps coordinates of all the cities (no subscription required) Documentation : https://nominatim.org/release-docs/develop/api/Search/

*   Use https://openweathermap.org/appid (you have to subscribe to get a free apikey) and https://openweathermap.org/api/one-call-api to get some information about the weather for the 35 cities and put it in a DataFrame

*   Determine the list of cities where the weather will be the nicest within the next 7 days For example, you can use the values of daily.pop and daily.rain to compute the expected volume of rain within the next 7 days... But it's only an example, actually you can have different opinions on a what a nice weather would be like 😎 Maybe the most important criterion for you is the temperature or humidity, so feel free to change the rules !

*   Save all the results in a `.csv` file, you will use it later 😉 You can save all the informations that seem important to you ! Don't forget to save the name of the cities, and also to create a column containing a unique identifier (id) of each city (this is important for what's next in the project)

*   Use plotly to display the best destinations on a map

### Scrape Booking.com 

Since BookingHoldings doesn't have aggregated databases, it will be much faster to scrape data directly from booking.com 

You can scrap as many information asyou want, but we suggest that you get at least:

*   hotel name,
*   Url to its booking.com page,
*   Its coordinates: latitude and longitude
*   Score given by the website users
*   Text description of the hotel


### Create your data lake using S3 

Once you managed to build your dataset, you should store into S3 as a csv file. 

### ETL 

Once you uploaded your data onto S3, it will be better for the next data analysis team to extract clean data directly from a Data Warehouse. Therefore, create a SQL Database using AWS RDS, extract your data from S3 and store it in your newly created DB. 

# Imports

In [195]:
import requests
import pandas as pd
import json
import time
from datetime import datetime
import plotly.express as px
import kaleido

In [2]:
# The list of cities we want to focus on

cities = ["Mont Saint Michel",
"St Malo",
"Bayeux",
"Le Havre",
"Rouen",
"Paris",
"Amiens",
"Lille",
"Strasbourg",
"Chateau du Haut Koenigsbourg",
"Colmar",
"Eguisheim",
"Besancon",
"Dijon",
"Annecy",
"Grenoble",
"Lyon",
"Gorges du Verdon",
"Bormes les Mimosas",
"Cassis",
"Marseille",
"Aix en Provence",
"Avignon",
"Uzes",
"Nimes",
"Aigues Mortes",
"Saintes Maries de la mer",
"Collioure",
"Carcassonne",
"Ariege",
"Toulouse",
"Montauban",
"Biarritz",
"Bayonne",
"La Rochelle"]

### Coordinate request

In [17]:
#Test API request with the city of Paris

url = "https://nominatim.openstreetmap.org/search"
payload = {'city' : 'Paris','country' : 'France','format':'json'}

gps_coord_test = requests.get(url, params = payload)
print(gps_coord_test)
print(gps_coord_test.json())

<Response [200]>
[{'place_id': 82615589, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'relation', 'osm_id': 7444, 'lat': '48.8588897', 'lon': '2.3200410217200766', 'class': 'boundary', 'type': 'administrative', 'place_rank': 15, 'importance': 0.8317101715588673, 'addresstype': 'suburb', 'name': 'Paris', 'display_name': 'Paris, Île-de-France, France métropolitaine, France', 'boundingbox': ['48.8155755', '48.9021560', '2.2241220', '2.4697602']}, {'place_id': 82297359, 'licence': 'Data © OpenStreetMap contributors, ODbL 1.0. http://osm.org/copyright', 'osm_type': 'relation', 'osm_id': 71525, 'lat': '48.8534951', 'lon': '2.3483915', 'class': 'boundary', 'type': 'administrative', 'place_rank': 12, 'importance': 0.8317101715588673, 'addresstype': 'city', 'name': 'Paris', 'display_name': 'Paris, Île-de-France, France métropolitaine, France', 'boundingbox': ['48.8155755', '48.9021560', '2.2241220', '2.4697602']}, {'place_id': 83151333, 'licenc

In [8]:
# Test getting coordinates

test_paris_lat = gps_coord_test.json()[0]['lat']
test_paris_lon = gps_coord_test.json()[0]['lon']
print(f"Latitude : {test_paris_lat}, Longitude : {test_paris_lon}")

Latitude : 48.8588897, Longitude : 2.3200410217200766


In [19]:
# Creating city coordinates df

cities_coord = []
url = "https://nominatim.openstreetmap.org/search"

for city in cities :
    payload = {'city' : city , 'country' : 'France', 'format':'json'}
    request_coord = requests.get(url, params = payload)
    request_coord_json = request_coord.json()
    if request_coord_json:
        lat = request_coord_json[0]['lat']
        lon = request_coord_json[0]['lon']
        name = request_coord_json[0]['name']
                
        cities_coord.append({"name": name, "lat": lat, "lon": lon})



In [23]:
print(cities_coord)
df_coord = pd.DataFrame(cities_coord)
df_coord.head()


[{'name': 'Mont Saint-Michel', 'lat': '48.6359541', 'lon': '-1.511459954959514'}, {'name': 'Saint-Malo', 'lat': '48.649518', 'lon': '-2.0260409'}, {'name': 'Bayeux', 'lat': '49.2764624', 'lon': '-0.7024738'}, {'name': 'Le Havre', 'lat': '49.4938975', 'lon': '0.1079732'}, {'name': 'Rouen', 'lat': '49.4404591', 'lon': '1.0939658'}, {'name': 'Paris', 'lat': '48.8588897', 'lon': '2.3200410217200766'}, {'name': 'Amiens', 'lat': '49.8941708', 'lon': '2.2956951'}, {'name': 'Lille', 'lat': '50.6365654', 'lon': '3.0635282'}, {'name': 'Strasbourg', 'lat': '48.584614', 'lon': '7.7507127'}, {'name': 'Château du Haut-Kœnigsbourg', 'lat': '48.2495226', 'lon': '7.3454923'}, {'name': 'Colmar', 'lat': '48.0777517', 'lon': '7.3579641'}, {'name': 'Eguisheim', 'lat': '48.0447968', 'lon': '7.3079618'}, {'name': 'Besançon', 'lat': '47.2380222', 'lon': '6.0243622'}, {'name': 'Dijon', 'lat': '47.3215806', 'lon': '5.0414701'}, {'name': 'Annecy', 'lat': '45.8992348', 'lon': '6.1288847'}, {'name': 'Grenoble', 'l

,name,lat,lon
0,Mont Saint-Michel,48.6359541,-1.511459954959514
1,Saint-Malo,48.649518,-2.0260409
2,Bayeux,49.2764624,-0.7024738
3,Le Havre,49.4938975,0.1079732
4,Rouen,49.4404591,1.0939658


### Weather data request

In [32]:
# Query meteo API to get the weather from the cities_coord list

#API Key to set in environment
appid = "***"

#Iterate through cities_coord to extract weather data
cities_weather = []

for city in range(len(df_coord)):

  url = "https://api.openweathermap.org/data/2.5/forecast"
  payload = {
           "lat": df_coord.loc[city, 'lat'],
           "lon":df_coord.loc[city,'lon'],
           "exclude":"current,minutely,hourly,alerts",
           "units":"metric",
           "appid":"3980a5f4b8c014468102d19a16bf8c86"
  }

  request_weather = requests.get(url, params=payload)
  request_weather_json = request_weather.json()
  cities_weather.append(request_weather_json)

In [70]:
len(cities_weather)

35

In [160]:
#Nouvelle boucle permettant d'itérer sur les différentes dates
cities_weather_clean = []

for city_weather in cities_weather:
    lat = city_weather["city"]["coord"]["lat"]
    lon = city_weather["city"]["coord"]["lon"]
    
    for forecast in city_weather["list"]:
        pop = forecast["pop"]
        temp = forecast["main"]["temp"]
        timestamp = forecast["dt"]
        date = datetime.fromtimestamp(timestamp)
        date_str = datetime.strftime(date, "%Y-%m-%d %H:%M:%S")

        cities_weather_clean.append({"lat":lat, "lon":lon, "pop":pop, "temp":temp, "date":date})



In [165]:
#check
print(len(cities_weather_clean))

#New df with weather info
df_weather = pd.DataFrame(cities_weather_clean)
df_weather_final = df_weather.groupby(['lat', 'lon', pd.Grouper(key='date', freq='D')]).agg({'temp': 'mean', 'pop': 'mean'}).reset_index()
df_weather_final["date"] = df_weather_final["date"].dt.date
df_weather_final["temp"] = df_weather_final["temp"].round(2)
df_weather_final["pop"] = df_weather_final["pop"].round(2)
df_weather_final.head()
df_weather_final.to_csv("weather_cities.csv")


1400


,lat,lon,date,temp,pop
0,42.5251,3.0832,2024-07-07,24.21,0.00
1,42.5251,3.0832,2024-07-08,23.39,0.02
2,42.5251,3.0832,2024-07-09,24.91,0.00
3,42.5251,3.0832,2024-07-10,25.35,0.00
4,42.5251,3.0832,2024-07-11,27.31,0.00


In [168]:
# Display map

fig = px.scatter_mapbox(df_weather_final,
                  lon='lon',
                  lat='lat',
                  color='pop',
                  size='temp',
                  hover_name='temp',
                  mapbox_style='open-street-map',
                  animation_group='date',
                  animation_frame='date',
                  title="Weather forecast for the cities targetted in the next 7 days",
                  height=600,
                  color_continuous_scale='viridis_r')
fig.update_layout(
    margin=dict(l=0, r=0, t=40, b=0),
    mapbox=dict(
        center=dict(lat=df_weather_final['lat'].mean(), lon=df_weather_final['lon'].mean()),
        zoom=4
    )
)

fig.write_html(r"G:\Mon Drive\Fichiers\2.Scolarité\1. Jedha_Data_Science\CERTIF_PROJECTS\ML_Engineer_Certification_Projects\02_DATA_COLLECTION_Kayak\Interactive_weather_map.html")

fig.show()

In [178]:
# Top 5 cities

df_top_weather = df_weather_final.copy()
df_top_weather["score"] = df_top_weather["temp"] - (df_top_weather["temp"]*df_top_weather["pop"])
df_top_weather = df_top_weather.groupby(["lat","lon"]).agg({"score":"mean"}).reset_index()
df_top_weather.head()

,lat,lon,score
0,42.5251,3.0832,24.983700
1,42.8458,1.6068,18.921083
2,43.1507,6.3419,23.940000
3,43.2130,2.3491,19.896500
4,43.2140,5.5396,23.673183


In [182]:
best_cities = df_top_weather.sort_values(by="score", ascending=False)
best_cities = best_cities.head(5)
best_cities

,lat,lon,score
9,43.5298,5.4475,25.257533
0,42.5251,3.0832,24.983700
5,43.2962,5.3700,24.194083
2,43.1507,6.3419,23.940000
13,43.8374,4.3601,23.888550


In [199]:
# Top 5 cities map

fig_2 = px.scatter_mapbox(best_cities,
                  lon='lon',
                  lat='lat',
                  color='score',
                  size='score',
                  mapbox_style='open-street-map',
                  title="Weather forecast for the TOP 5 cities",
                  height=600,)
fig_2.update_layout(
    margin=dict(l=0, r=0, t=40, b=0),
    mapbox=dict(
        center=dict(lat=46.232193, lon= 2.209667),
        zoom=5
    )
)
fig_2.write_image(r"G:\Mon Drive\Fichiers\2.Scolarité\1. Jedha_Data_Science\CERTIF_PROJECTS\ML_Engineer_Certification_Projects\02_DATA_COLLECTION_Kayak/Top_5_cities_weather_map.png")
fig_2.show()

# Hotel scrapping